Reference:

1. (Article) Accelerate LLAMA & LangChain with Local GPU https://lolevsky.medium.com/running-locally-llama-and-langchain-accelerated-by-gpu-a52a2fd72d79
2. (Video) How to install PyTorch on Ubuntu 22.04 with Nvidia graphics card https://www.youtube.com/watch?v=c0Z_ItwzT5o

Here, this instruction guide is specifically for LINUX(UBUNTU) operation system computer. 

When testing the below procedure, Ubuntu 22.04 LTS is used. 

## Step 0: Allow your UBUNTU running sudo commands without password

This step ensure all code blocks below could be executed smoothly. 

1. `sudo visudo` to open related file

2. under `%sudo ALL=(ALL:ALL) ALL`, add one more line: `username ALL=(ALL) NOPASSWD:ALL`. Replace username with your username

3. Quit the editor with `ctrl + X`

Then here we go:

In [2]:
#!/bin/bash
!sudo apt update

Get:1 file:/var/cudnn-local-repo-ubuntu2004-8.8.1.3  InRelease [1,572 B]
Get:1 file:/var/cudnn-local-repo-ubuntu2004-8.8.1.3  InRelease [1,572 B]
Hit:2 http://hk.archive.ubuntu.com/ubuntu focal InRelease                      
Hit:3 http://hk.archive.ubuntu.com/ubuntu focal-updates InRelease              
Hit:4 http://hk.archive.ubuntu.com/ubuntu focal-backports InRelease            
Hit:5 https://packages.microsoft.com/repos/code stable InRelease               
Hit:6 https://dl.google.com/linux/chrome/deb stable InRelease                  
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://hp.archive.canonical.com focal InRelease                      
Hit:9 http://security.ubuntu.com/ubuntu focal-security InRelease    
Hit:10 http://oem.archive.canonical.com focal InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
111 packages can be upgraded. Run 'apt list --upgradable' to see 

## Step 1: Setup a proper python version

Different version of CUDA support a limited range of python. 
So we need to check the release.md to get to know: [release page](https://github.com/pytorch/pytorch/blob/main/RELEASE.md#release-compatibility-matrix)

### method 1: use `apt` to install python

In [9]:
!sudo apt install python3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3 is already the newest version (3.8.2-0ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 111 not upgraded.


In [11]:
!python3 --version

Python 3.8.10


Then we install pip:

In [12]:
!sudo apt install python3-pip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-pip is already the newest version (20.0.2-5ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 111 not upgraded.


Then verify pip installation:

In [14]:
!pip3 --version

pip 20.0.2 from /usr/lib/python3/dist-packages/pip (python 3.8)


### method 2: use `pyenv` to overwrite which specify python version we want to use

In the following, we use python script to add "pyenv", "pyenv-virtual" command to the bashrc:

In [7]:
import subprocess
import sys
import os

def run_command(command):
    """Run a command in the subprocess and handle exceptions."""
    try:
        result = subprocess.run(
            command,
            shell=True,
            check=True,
            executable="/bin/bash",
            text=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        print(result.stdout)
        return result
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e.stderr}", file=sys.stderr)
        sys.exit(e.returncode)

# Update package list
print("Updating package list...")
run_command("sudo apt update")

# Install dependencies for pyenv and Python build
print("Installing dependencies...")
dependencies = [
    'make', 'build-essential', 'libssl-dev', 'zlib1g-dev', 'libbz2-dev',
    'libreadline-dev', 'libsqlite3-dev', 'wget', 'curl', 'llvm',
    'libncursesw5-dev', 'xz-utils', 'tk-dev', 'libxml2-dev', 'libxmlsec1-dev',
    'libffi-dev', 'liblzma-dev', 'git'
]
run_command(f"sudo apt install -y {' '.join(dependencies)}")

# Install pyenv
print("Cloning pyenv repository...")
pyenv_root = os.path.expanduser("~/.pyenv")
if not os.path.exists(pyenv_root):
    run_command(f"git clone https://github.com/pyenv/pyenv.git {pyenv_root}")

# Add pyenv to PATH and enable shims and autocompletion
print("Configuring the environment for pyenv...")
with open(os.path.expanduser("~/.bashrc"), 'a') as bashrc:
    bashrc_lines = [
        'export PYENV_ROOT="$HOME/.pyenv"',
        'export PATH="$PYENV_ROOT/bin:$PATH"',
        'eval "$(pyenv init --path)"',
        'eval "$(pyenv init -)"'
    ]
    for line in bashrc_lines:
        bashrc.write(f"{line}\n")

# Initialize pyenv in this script's environment
os.environ["PYENV_ROOT"] = pyenv_root
os.environ["PATH"] += f":{pyenv_root}/bin"
source_pyenv = 'source ~/.bashrc && eval "$(pyenv init --path)" && eval "$(pyenv init -)"'
run_command(source_pyenv)

# Install pyenv-virtualenv as a pyenv plugin
print("Installing pyenv-virtualenv...")
pyenv_virtualenv_dir = os.path.join(pyenv_root, "plugins/pyenv-virtualenv")
if not os.path.exists(pyenv_virtualenv_dir):
    run_command(f"git clone https://github.com/pyenv/pyenv-virtualenv.git {pyenv_virtualenv_dir}")

# Update the .bashrc file for pyenv-virtualenv
with open(os.path.expanduser("~/.bashrc"), 'a') as bashrc:
    bashrc.write('eval "$(pyenv virtualenv-init -)"\n')

print("""
Installation complete! pyenv-virtualenv is installed.
To enable it, restart your shell or run the following command:
source ~/.bashrc
""")

# Now, let's apply the changes to the current shell session:
run_command("source ~/.bashrc")

# Check if pyenv is now recognized
!pyenv --version
!pyenv virtualenv --version

Updating package list...
Get:1 file:/var/cudnn-local-repo-ubuntu2004-8.8.1.3  InRelease [1,572 B]
Get:1 file:/var/cudnn-local-repo-ubuntu2004-8.8.1.3  InRelease [1,572 B]
Hit:2 https://packages.microsoft.com/repos/code stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:5 http://hk.archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://hk.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://hk.archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://oem.archive.canonical.com focal InRelease
Hit:9 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:10 http://hp.archive.canonical.com focal InRelease
Reading package lists...
Building dependency tree...
Reading state information...
107 packages can be upgraded. Run 'apt list --upgradable' to see them.

Installing dependencies...
Reading package lists...
Building dependency tree...
Reading s

### Installation verification

If `pyenv` and `pyenv-virtualenv` version are properly displayed in the last block, that means they are properly installed, 

Uae `pyenv` to select a specific python version to download:

In [25]:
!pyenv install 3.11.7 -s

-> https://www.python.org/ftp/python/3.11.7/Python-3.11.7.tar.xz
Installing Python-3.11.7...
Installed Python-3.11.7 to /home/researcher/.pyenv/versions/3.11.7


Here we list all possible python version managed by pyenv:

In [3]:
!pyenv shims

/home/researcher/.pyenv/shims/2to3
/home/researcher/.pyenv/shims/2to3-3.11
/home/researcher/.pyenv/shims/idle
/home/researcher/.pyenv/shims/idle3
/home/researcher/.pyenv/shims/idle3.11
/home/researcher/.pyenv/shims/pip
/home/researcher/.pyenv/shims/pip3
/home/researcher/.pyenv/shims/pip3.11
/home/researcher/.pyenv/shims/pydoc
/home/researcher/.pyenv/shims/pydoc3
/home/researcher/.pyenv/shims/pydoc3.11
/home/researcher/.pyenv/shims/python
/home/researcher/.pyenv/shims/python3
/home/researcher/.pyenv/shims/python3.11
/home/researcher/.pyenv/shims/python3.11-config
/home/researcher/.pyenv/shims/python3.11-gdb.py
/home/researcher/.pyenv/shims/python3-config
/home/researcher/.pyenv/shims/python-config


Now, we set the global python version to be 3.11.7

(I dont want to setup 'pyenv virtualenv' because it seems complicated to manage virtual environemnt of python)

In [10]:
!pyenv global 3.11.7
!python --version

Python 3.11.7


### Change your python kernel in VSCODE

Now, a new python environment under `pyenv` folder is created. 
To be consistent, when using jupyter notebook, we should also choose the same instance. 




## Step 2: Make sure Nvidia drivers are installed

You should have a computer that is using Nvidia graphic card.

To check which version of nvidia driver is in the computer:

In [3]:
!dpkg -l | grep nvidia-driver

ii  nvidia-driver-530                          535.129.03-0ubuntu0.20.04.1           amd64        Transitional package for nvidia-driver-535
ii  nvidia-driver-535                          535.129.03-0ubuntu0.20.04.1           amd64        NVIDIA driver metapackage


We can install nvidia driver automatically: 

In [6]:
!sudo apt update && sudo apt upgrade -y
!sudo apt --fix-broken install
!sudo apt autoremove -y
!sudo dpkg --configure -a
!sudo apt purge nvidia-* -y
!sudo apt autoremove -y
!sudo apt autoclean
!sudo ubuntu-drivers autoinstall

Get:1 file:/var/cudnn-local-repo-ubuntu2004-8.8.1.3  InRelease [1,572 B]
Get:1 file:/var/cudnn-local-repo-ubuntu2004-8.8.1.3  InRelease [1,572 B]
Hit:2 http://hk.archive.ubuntu.com/ubuntu focal InRelease                      
Hit:3 http://hk.archive.ubuntu.com/ubuntu focal-updates InRelease              
Hit:4 http://hk.archive.ubuntu.com/ubuntu focal-backports InRelease            
Hit:5 https://packages.microsoft.com/repos/code stable InRelease               
Hit:6 https://dl.google.com/linux/chrome/deb stable InRelease                  
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://oem.archive.canonical.com focal InRelease                         
Hit:9 http://hp.archive.canonical.com focal InRelease                          
Hit:10 http://security.ubuntu.com/ubuntu focal-security InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
107 packages can be upgraded. Run 'apt l

Then refresh driver list:

In [10]:
!dpkg -l | grep nvidia-driver

ii  nvidia-driver-535                          535.161.07-0ubuntu0.20.04.1           amd64        NVIDIA driver metapackage


To apply those changes, we need to reboot our computer:

In [ ]:
!sudo reboot

We can also check which version of driver is in user:

In [4]:
!nvidia-smi

Tue Mar  5 03:19:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   38C    P0              N/A /  80W |     17MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Step 3: Download and install CUDA

Get to know your PC information:

In [1]:
!uname -m
!dpkg --print-architecture
!lsb_release -a

x86_64
amd64
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.6 LTS
Release:	20.04
Codename:	focal


Go to [CUDA Developer CUDA download page](https://developer.nvidia.com/cuda-downloads?target_os=Linux&target_arch=x86_64&Distribution=Ubuntu) to get download link.

I choose `deb (local)` because it is the easiest and can be fully controlled by apt.

Please update the following chain of commands to be what the website provided. 

In [2]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
!sudo mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/12.3.2/local_installers/cuda-repo-ubuntu2004-12-3-local_12.3.2-545.23.08-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu2004-12-3-local_12.3.2-545.23.08-1_amd64.deb
!sudo cp /var/cuda-repo-ubuntu2004-12-3-local/cuda-*-keyring.gpg /usr/share/keyrings/
!sudo apt-get update
!sudo apt-get -y install cuda-toolkit-12-3

--2024-03-05 03:34:20--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.39.144
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu2004.pin’

cuda-ubuntu2004.pin 100%[===================>]     190  --.-KB/s    in 0s      

2024-03-05 03:34:20 (6.61 MB/s) - ‘cuda-ubuntu2004.pin’ saved [190/190]

--2024-03-05 03:34:21--  https://developer.download.nvidia.com/compute/cuda/12.3.2/local_installers/cuda-repo-ubuntu2004-12-3-local_12.3.2-545.23.08-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.39.144
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 200 O

Now, set `nvcc` location into `.bashrc`:

In [5]:
import os

# Set the PATH and LD_LIBRARY_PATH for the CUDA Toolkit
os.environ['PATH'] += os.pathsep + '/usr/local/cuda-12.3/bin'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-12.3/lib64' + os.pathsep + os.getenv('LD_LIBRARY_PATH', '')

Then, we try to verify installation:

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Nov_22_10:17:15_PST_2023
Cuda compilation tools, release 12.3, V12.3.107
Build cuda_12.3.r12.3/compiler.33567101_0


Find CUDA directory in the PC:

In [9]:
!find /usr/local -type d -name 'samples' 2>/dev/null

/usr/local/cuda-12.3/gds/samples
/usr/local/cuda-12.3/extras/CUPTI/samples


`gds` stands for GPU direct storage. And GDS is a technology that allows applications to bypass the CPU when accessing storage directly from the GPU memory, which reduce latency and increase bandwidth for data transfer between the GPU and storage.

(We might add a block that demonstrate how to run sample CUDA project below)

## Step 4: Download and install PyTorch

- Go to https://pytorch.org/
- Input your PC specifications to generate the command for downloading PyTorch package:

- Linux
- Pip
- Python
- CUDA 12.1 (Not exactly match, but choose the closest version)

In [12]:
!pip3 install torch torchvision torchaudio

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/2c/df/5810707da6f2fd4be57f0cc417987c0fa16a2eecf0b1b71f82ea555dc619/torch-2.2.1-cp311-cp311-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/3a/49/12fc5188602c68a789a0fdaee63d176a71ad5c1e34d25aeb8554abe46089/torchvision-0.17.1-cp311-cp311-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/a6/57/ccebdda4db80e384166c70d8645fa998637051b3b19aca1fd8de80602afb/torchaudio-2.2.1-cp311-cp311-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/81/54/84d42a0bee35edba99dee7b59a8d4970eccdd44b99fe728ed912106fc781/filelock-3.13.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/f9/de/dc04a3ea60b22624b51c703a

Check installation

In [13]:
import torch
A = torch.rand(3)
print(A)

tensor([0.3760, 0.8427, 0.3414])


If the code block above has no problem. Then the installation is successful.

## Step 5: Check PyTorch has access to GPU

In [15]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA GeForce RTX 3080 Ti Laptop GPU
